## Deliverable 2. Create a Customer Travel Destinations Map.

In [12]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [13]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carnarvon,AU,-24.8667,113.6333,75.27,69,100,21.85,overcast clouds
1,1,Chuy,UY,-33.6971,-53.4616,75.27,39,3,16.82,clear sky
2,2,New Norfolk,AU,-42.7826,147.0587,52.72,74,4,1.95,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,60.46,59,75,17.27,broken clouds
4,4,Faanui,PF,-16.4833,-151.7500,81.23,75,100,22.88,overcast clouds


In [14]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 57
What is the maximum temperature you would like for your trip? 79


In [15]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(15)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carnarvon,AU,-24.8667,113.6333,75.27,69,100,21.85,overcast clouds
1,1,Chuy,UY,-33.6971,-53.4616,75.27,39,3,16.82,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,60.46,59,75,17.27,broken clouds
6,6,Atuona,PF,-9.8000,-139.0333,78.78,72,0,19.04,clear sky
7,7,Punta Arenas,CL,-53.1500,-70.9167,69.91,49,75,16.11,broken clouds
10,10,Potiskum,NG,11.7091,11.0694,71.56,20,0,10.67,clear sky
11,11,Saint-Philippe,RE,-21.3585,55.7679,78.39,80,99,15.50,overcast clouds
13,13,Vaini,TO,-21.2000,-175.2000,75.36,94,100,12.66,light rain
16,16,Hermanus,ZA,-34.4187,19.2345,67.89,89,9,8.19,clear sky
19,19,Avarua,CK,-21.2078,-159.7750,77.05,88,85,10.36,overcast clouds


In [16]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                276
City                   276
Country                270
Lat                    276
Lng                    276
Max Temp               276
Humidity               276
Cloudiness             276
Wind Speed             276
Current Description    276
dtype: int64

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(subset = ["Country"], inplace=True)
preferred_cities_df.count()

City_ID                270
City                   270
Country                270
Lat                    270
Lng                    270
Max Temp               270
Humidity               270
Cloudiness             270
Wind Speed             270
Current Description    270
dtype: int64

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carnarvon,AU,75.27,overcast clouds,-24.8667,113.6333,
1,Chuy,UY,75.27,clear sky,-33.6971,-53.4616,
3,Ushuaia,AR,60.46,broken clouds,-54.8000,-68.3000,
6,Atuona,PF,78.78,clear sky,-9.8000,-139.0333,
7,Punta Arenas,CL,69.91,broken clouds,-53.1500,-70.9167,
10,Potiskum,NG,71.56,clear sky,11.7091,11.0694,
11,Saint-Philippe,RE,78.39,overcast clouds,-21.3585,55.7679,
13,Vaini,TO,75.36,light rain,-21.2000,-175.2000,
16,Hermanus,ZA,67.89,clear sky,-34.4187,19.2345,
19,Avarua,CK,77.05,overcast clouds,-21.2078,-159.7750,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carnarvon,AU,75.27,overcast clouds,-24.8667,113.6333,Hospitality Carnarvon
1,Chuy,UY,75.27,clear sky,-33.6971,-53.4616,Nuevo Hotel Plaza
3,Ushuaia,AR,60.46,broken clouds,-54.8000,-68.3000,Albatross Hotel
6,Atuona,PF,78.78,clear sky,-9.8000,-139.0333,Villa Enata
7,Punta Arenas,CL,69.91,broken clouds,-53.1500,-70.9167,Hotel Hain
10,Potiskum,NG,71.56,clear sky,11.7091,11.0694,Yobe NYSC Orietation Camp Dazigau
11,Saint-Philippe,RE,78.39,overcast clouds,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
13,Vaini,TO,75.36,light rain,-21.2000,-175.2000,Keleti Beach Resort
16,Hermanus,ZA,67.89,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
19,Avarua,CK,77.05,overcast clouds,-21.2078,-159.7750,Paradise Inn


In [57]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df.count()

City                   246
Country                246
Max Temp               246
Current Description    246
Lat                    246
Lng                    246
Hotel Name             246
dtype: int64

In [58]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [65]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [66]:
# 11a. Add a marker layer for each city to the map. 
# Heatmap of temperature for vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))